In [1]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

In [2]:
investment_path = 'investments_VC.csv'
acquisition_path = 'Acquisitions.csv'

df_investments = pd.read_csv(investment_path, encoding='latin1')
df_acquisitions = pd.read_csv(acquisition_path)

In [3]:
# Clean column names
df_investments.columns = df_investments.columns.str.strip()

# Clean the 'funding_total_usd' column
df_investments['funding_total_usd'] = df_investments['funding_total_usd'].str.replace(',', '').str.strip()

# Non numeric values to be turned into NaN
df_investments['funding_total_usd'] = pd.to_numeric(df_investments['funding_total_usd'], errors='coerce')

print("Cleaned 'funding_total_usd' data type:")
print(df_investments['funding_total_usd'].dtype)

Cleaned 'funding_total_usd' data type:
float64


In [4]:
def clean_company_name(name):
    if not isinstance(name, str):
        return None
    name = name.lower()
    # Remove common suffixes
    name = re.sub(r'\s+(inc|llc|corporation|corp|ltd|co)\.?$', '', name)
    # Remove punctuation
    name = re.sub(r'[.,;:]', '', name)
    return name.strip()

# Apply the cleaning function to both dataframes
df_investments['clean_name'] = df_investments['name'].apply(clean_company_name)
df_acquisitions['clean_acquired_company'] = df_acquisitions['Acquired Company'].apply(clean_company_name)

print("Example of cleaned names:")
print(df_investments[['name', 'clean_name']].head())

Example of cleaned names:
                 name          clean_name
0            #waywire            #waywire
1  &TV Communications  &tv communications
2   'Rock' Your Paper   'rock' your paper
3   (In)Touch Network   (in)touch network
4  -R- Ranch and Mine  -r- ranch and mine


In [5]:
df_merged = pd.merge(
    df_investments,
    df_acquisitions[['clean_acquired_company', 'Deal announced on']],
    left_on='clean_name',
    right_on='clean_acquired_company',
    how='left'
)

df_merged = df_merged.drop('clean_acquired_company', axis=1)
df_merged = df_merged.drop_duplicates(subset=['permalink'])

print(f"Original investments count: {len(df_investments)}")
print(f"Merged dataframe count: {len(df_merged)}")
print(f"Number of successful matches (acquisitions): {df_merged['Deal announced on'].notna().sum()}")

Original investments count: 54294
Merged dataframe count: 49437
Number of successful matches (acquisitions): 662


In [6]:
# Convert date columns to datetime objects
df_merged['last_funding_at'] = pd.to_datetime(df_merged['last_funding_at'], errors='coerce')
df_merged['Deal announced on'] = pd.to_datetime(df_merged['Deal announced on'], errors='coerce')

# Calculate the time difference in days
df_merged['days_to_acquisition'] = (df_merged['Deal announced on'] - df_merged['last_funding_at']).dt.days

# Create the target label
df_merged['acquired_within_1_year'] = 0

# Set the label to 1 only if the acquisition happened between 0 and 365 days after the last funding
df_merged.loc[
    (df_merged['days_to_acquisition'] >= 0) & (df_merged['days_to_acquisition'] <= 365),
    'acquired_within_1_year'
] = 1

print("Target Label Distribution:")
print(df_merged['acquired_within_1_year'].value_counts())

Target Label Distribution:
acquired_within_1_year
0    49346
1       91
Name: count, dtype: int64


In [7]:
# Select relevant columns for modeling
final_cols = [
    'permalink',
    'name',
    'market',
    'funding_total_usd',
    'status',
    'country_code',
    'funding_rounds',
    'founded_at',
    'last_funding_at',
    'seed',
    'venture',
    'round_A',
    'round_B',
    'round_C',
    'acquired_within_1_year' # Our target variable
]

# Create the final dataframe, dropping rows with critical missing data for our model
df_final = df_merged[final_cols].dropna(subset=['market', 'funding_total_usd', 'last_funding_at'])

output_path = 'cleaned_data_with_target.csv'
df_final.to_csv(output_path, index=False)

print(f"Processed data saved to {output_path}")
print(f"Final dataset shape: {df_final.shape}")

Processed data saved to cleaned_data_with_target.csv
Final dataset shape: (38398, 15)


In [8]:
import pandas as pd

df_check = pd.read_csv('cleaned_data_with_target.csv')

print("--- DataFrame Info ---")
df_check.info()

print("\n--- DataFrame Head ---")
df_check.head()

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38398 entries, 0 to 38397
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   permalink               38398 non-null  object 
 1   name                    38397 non-null  object 
 2   market                  38398 non-null  object 
 3   funding_total_usd       38398 non-null  float64
 4   status                  37586 non-null  object 
 5   country_code            35243 non-null  object 
 6   funding_rounds          38398 non-null  float64
 7   founded_at              31037 non-null  object 
 8   last_funding_at         38398 non-null  object 
 9   seed                    38398 non-null  float64
 10  venture                 38398 non-null  float64
 11  round_A                 38398 non-null  float64
 12  round_B                 38398 non-null  float64
 13  round_C                 38398 non-null  float64
 14  acquired_within

,permalink,name,market,funding_total_usd,status,country_code,funding_rounds,founded_at,last_funding_at,seed,venture,round_A,round_B,round_C,acquired_within_1_year
0,/organization/waywire,#waywire,News,1750000.0,acquired,USA,1.0,2012-06-01,2012-06-30,1750000.0,0.0,0.0,0.0,0.0,0
1,/organization/tv-communications,&TV Communications,Games,4000000.0,operating,USA,2.0,NaN,2010-09-23,0.0,4000000.0,0.0,0.0,0.0,0
2,/organization/rock-your-paper,'Rock' Your Paper,Publishing,40000.0,operating,EST,1.0,2012-10-26,2012-08-09,40000.0,0.0,0.0,0.0,0.0,0
3,/organization/in-touch-network,(In)Touch Network,Electronics,1500000.0,operating,GBR,1.0,2011-04-01,2011-04-01,1500000.0,0.0,0.0,0.0,0.0,0
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,Tourism,60000.0,operating,USA,2.0,2014-01-01,2014-09-26,0.0,0.0,0.0,0.0,0.0,0
